In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
cle = pd.read_csv('metadata/cle_metadata.csv')
vir = pd.read_csv('metadata/vir_metadata.csv')
hun = pd.read_csv('metadata/hun_metadata.csv')
swi = pd.read_csv('metadata/swi_metadata.csv')

In [3]:
data = pd.concat([cle,vir,hun,swi])
data

0         1         2         3    4    5         6    7  \
0    0.000000  0.010828  0.770814  0.603649  0.0  0.0  0.745914  0.0   
1    0.000000  0.000000  0.710489  0.546340  0.0  0.0  0.609842  0.0   
2    0.000000  0.177793  0.822739  0.714406  0.0  0.0  0.903484  0.0   
3    0.000000  0.000000  0.739092  0.573498  0.0  0.0  0.809695  0.0   
4    0.000000  0.130649  0.627133  0.457336  0.0  0.0  0.494202  0.0   
..        ...       ...       ...       ...  ...  ...       ...  ...   
118  0.386348  0.000000  0.000000  0.447409  0.0  0.0  0.000000  0.0   
119  0.276377  0.000000  0.000000  0.336829  0.0  0.0  0.000000  0.0   
120  0.285732  0.000000  0.000000  0.382396  0.0  0.0  0.000000  0.0   
121  0.328392  0.000000  0.000000  0.439503  0.0  0.0  0.000000  0.0   
122  0.254993  0.000000  0.000000  0.343928  0.0  0.0  0.000000  0.0   

            8    9  ...       119       120       121       122       123  \
0    0.193387  0.0  ...  0.000000  0.000000  0.000000  0.138432  0.293800   
1    0.249503  0.0  ...  0.072369  0.000000  0.000000  0.392468  0.223925   
2    0.000000  0.0  ...  0.000000  0.000000  0.000000  0.021255  0.009928   
3    0.123018  0.0  ...  0.000000  0.000000  0.000000  0.219051  0.181945   
4    0.322922  0.0  ...  0.000000  0.000000  0.000000  0.307420  0.298971   
..        ...  ...  ...       ...       ...       ...       ...       ...   
118  0.392385  0.0  ...  0.000000  0.472914  0.627784  0.000000  0.000000   
119  0.327667  0.0  ...  0.000000  0.338120  0.435492  0.000000  0.000000   
120  0.344254  0.0  ...  0.000000  0.371470  0.471161  0.000000  0.000000   
121  0.338543  0.0  ...  0.000000  0.401438  0.545498  0.000000  0.000000   
122  0.317490  0.0  ...  0.000000  0.320704  0.477940  0.000000  0.000000   

          124  125       126       127   num  
0    0.242231  0.0  0.213967  0.243880  0.00  
1    0.421081  0.0  0.423841  0.408610  0.00  
2    0.000000  0.0  0.020860  0.010512  0.00  
3    0.163801  0.0  0.140269  0.167836  0.00  
4    0.389081  0.0  0.363960  0.324965  0.00  
..        ...  ...       ...       ...   ...  
118  0.408348  0.0  0.420346  0.000000  0.75  
119  0.344342  0.0  0.238326  0.000000  0.50  
120  0.358984  0.0  0.295315  0.000000  1.00  
121  0.376631  0.0  0.356814  0.000000  0.25  
122  0.347801  0.0  0.257057  0.000000  0.25  

[920 rows x 129 columns]

In [4]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Y_train_binary = Y_train.apply(lambda x: 1 if x > 0 else 0)
Y_test_binary = Y_test.apply(lambda x: 1 if x > 0 else 0)

In [7]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

# define the model architecture
model = Sequential()
model.add(Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(128,1)))
model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
model.fit(X_train, Y_train_binary, epochs=1000, batch_size=32,callbacks=[callback])
Y_pred = model.predict(X_test).argmax(axis=1)
    
cm = confusion_matrix(Y_pred, Y_test_binary)
print(cm)
print(classification_report(Y_test_binary, Y_pred))

Epoch 1/1000
20/20 [==============================] - 2s 71ms/step - loss: 0.3629 - accuracy: 0.8214
Epoch 2/1000
20/20 [==============================] - 1s 71ms/step - loss: 0.3218 - accuracy: 0.8847
Epoch 3/1000
20/20 [==============================] - 1s 72ms/step - loss: 0.3228 - accuracy: 0.8750
Epoch 4/1000
20/20 [==============================] - 1s 70ms/step - loss: 0.3117 - accuracy: 0.8831
Epoch 5/1000
20/20 [==============================] - 1s 70ms/step - loss: 0.2896 - accuracy: 0.8945
Epoch 6/1000
20/20 [==============================] - 1s 70ms/step - loss: 0.3103 - accuracy: 0.8782
Epoch 7/1000
20/20 [==============================] - 1s 70ms/step - loss: 0.2987 - accuracy: 0.8847
Epoch 8/1000
20/20 [==============================] - 1s 70ms/step - loss: 0.2847 - accuracy: 0.8945
Epoch 9/1000
20/20 [==============================] - 1s 71ms/step - loss: 0.2943 - accuracy: 0.8929
Epoch 10/1000
20/20 [==============================] - 1s 72ms/step - loss: 0.2920 - accura